In [39]:
%load_ext autoreload
%autoreload 2

import torch
from torch.nn.functional import unfold, fold
from others.implementations import Convolution
from torch.nn.functional import conv2d

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Convolution

In [40]:
padding = 5
stride = 2
custom_conv = Convolution(3, 1, kernel_size=2, padding=padding, stride = stride)

inp = torch.ones((5,3,13,13))
inp[:,0,:,:] = 3
inp.requires_grad = True
out = custom_conv.forward(inp)
target = torch.zeros(out.size())

kernel = torch.ones((1,3,2,2))
kernel.requires_grad = True
kernel.retain_grad()
inp_padded = custom_conv.add_padding(inp, padding)
inp_padded = inp_padded.detach()
inp_padded.requires_grad = True
target_out = conv2d(inp_padded, kernel, stride=stride)
target_out.retain_grad()
loss = 0.5*((target-target_out)**2).sum()
loss.backward()
dl_dout = target_out.grad.detach()
dl_dw = kernel.grad.detach()
dl_dx = inp_padded.grad.detach()

In [41]:
#check out
(target_out - out).abs().sum()

tensor(0., grad_fn=<SumBackward0>)

In [42]:
#check dl_dx
(custom_conv.backward(dl_dout) - dl_dx).abs().sum()

tensor(0.)

In [43]:
#check dl_dw
(custom_conv.dl_dw - dl_dw).abs().sum()

tensor(0., grad_fn=<SumBackward0>)

## Up-sampling

In [399]:
from torch import nn
import others.implementations

In [400]:
scale_factor = 5
target_upsample = nn.Upsample(scale_factor = scale_factor, mode = 'nearest')
custom_upsample = others.implementations.Upsample(scale_factor = scale_factor)
inp = torch.ones((1,3,14,8))
inp.requires_grad = True
target_out = target_upsample.forward(inp)
target_out.retain_grad()
custom_out = custom_upsample.forward(inp)
target = torch.zeros((1,3,inp.shape[-2]*scale_factor, inp.shape[-1]*scale_factor))
loss = 0.5*((target-target_out)**2).sum()
loss.backward()


In [401]:
#check out
(target_out - custom_out).abs().sum()

tensor(0., grad_fn=<SumBackward0>)

In [402]:
#check dl_dx
(custom_upsample.backward(target_out)-inp.grad).abs().sum()

tensor(0., grad_fn=<SumBackward0>)